In [1]:
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import sklearn.linear_model as sklm 
import sklearn.decomposition as skdc
import sklearn.pipeline as skpl
%matplotlib inline

In [2]:
import Comet_Removal
%run Comet_Removal

print(len(data))

181


In [3]:

msk=np.random.rand(len(data))<0.7
training=data[msk]
test=data[~msk]
len(test)
len(training)


120

In [4]:
training.head()

,﻿Designation,Discovery Date YYYY-MM-DD,H (mag),MOID (AU),q (AU),Q (AU),period (yr),i (deg),PHA,Orbit Class
1,419624 (2010 SO16),09/17/2010,20.5,0.028,0.93,1.08,1.00,14.52,Y,Apollo
3,414746 (2010 EH20),03/06/2010,18.0,0.268,1.25,3.99,4.24,23.89,N,Amor
4,407324 (2010 OB101),07/18/2010,20.7,0.111,0.77,2.46,2.06,9.12,N,Apollo
6,395207 (2010 HQ80),04/25/2010,19.6,0.007,0.80,2.34,1.96,27.85,Y,Apollo
8,381989 (2010 HR80),04/28/2010,19.9,0.104,0.68,2.02,1.56,26.71,N,Apollo


In [5]:
#print(pd.get_dummies(training['Designation']).head(1))
#training['Designation_dummies'] = pd.get_dummies(training['Designation']).iloc[:,1]

In [6]:
y=training["PHA"]
x=training.iloc[:,2:7]
#x=train[:,"Designation":"PHA"]
#x=train["Designation","Discovery Date YYYY-MM-DD","H (mag)"]
x.head()



,H (mag),MOID (AU),q (AU),Q (AU),period (yr)
1,20.5,0.028,0.93,1.08,1.00
3,18.0,0.268,1.25,3.99,4.24
4,20.7,0.111,0.77,2.46,2.06
6,19.6,0.007,0.80,2.34,1.96
8,19.9,0.104,0.68,2.02,1.56


In [7]:
model = LogisticRegression()
model.fit(x,y)
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
model.coef_

array([[ 0.01399233, -1.61918404, -0.17378038, -0.42217922, -0.16409325]])

In [9]:
model.intercept_

array([ 0.03630652])

In [10]:
newX=test.iloc[:,2:7]

In [11]:
model.predict(newX)

array(['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'], dtype=object)

In [12]:
print(model.classes_)
print(model.predict_proba(newX))
k=model.predict_proba(newX)

['N' 'Y']
[[ 0.91336199  0.08663801]
 [ 0.7964289   0.2035711 ]
 [ 0.60703286  0.39296714]
 [ 0.79000204  0.20999796]
 [ 0.80860722  0.19139278]
 [ 0.68826528  0.31173472]
 [ 0.86726754  0.13273246]
 [ 0.88069133  0.11930867]
 [ 0.92795979  0.07204021]
 [ 0.90272446  0.09727554]
 [ 0.93595523  0.06404477]
 [ 0.91543053  0.08456947]
 [ 0.97223357  0.02776643]
 [ 0.95107704  0.04892296]
 [ 0.72568984  0.27431016]
 [ 0.92071518  0.07928482]
 [ 0.90994518  0.09005482]
 [ 0.70567497  0.29432503]
 [ 0.86127687  0.13872313]
 [ 0.95572385  0.04427615]
 [ 0.76245196  0.23754804]
 [ 0.76805453  0.23194547]
 [ 0.86539943  0.13460057]
 [ 0.93536383  0.06463617]
 [ 0.65646119  0.34353881]
 [ 0.91274636  0.08725364]
 [ 0.92848602  0.07151398]
 [ 0.96017243  0.03982757]
 [ 0.76743013  0.23256987]
 [ 0.73813324  0.26186676]
 [ 0.85281004  0.14718996]
 [ 0.80830745  0.19169255]
 [ 0.94076466  0.05923534]
 [ 0.94807423  0.05192577]
 [ 0.82815233  0.17184767]
 [ 0.64235616  0.35764384]
 [ 0.62562988  0.3

In [13]:
k=pd.DataFrame(k)
indx = k.idxmax(axis=1)
pred_class=model.classes_[indx]
pred_class

array(['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N',
       'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N'], dtype=object)

In [14]:
newy=test["PHA"]
newy.head()



0    Y
2    N
5    Y
7    Y
9    N
Name: PHA, dtype: object

In [15]:
#newy==pred_class

In [16]:
con=confusion_matrix(newy,pred_class)
con

array([[50,  0],
       [11,  0]])

In [17]:
print(sum(con))
print(sum(sum(con)))

[61  0]
61


In [18]:
print(classification_report(newy,pred_class))

             precision    recall  f1-score   support

          N       0.82      1.00      0.90        50
          Y       0.00      0.00      0.00        11

avg / total       0.67      0.82      0.74        61



C:\ANACONDA3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
